In [1]:
import os
import pickle
import numpy as np
import pandas as pd
import nltk, re, pprint
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
nltk.download('wordnet')
from nltk.stem.wordnet import WordNetLemmatizer
import sys
#sys.setdefaultencoding("utf-8")

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Curri\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Create list of stopwords from custom list and built-in library

In [59]:
from nltk.corpus import stopwords
ENGLISH_STOP_WORDS = set(stopwords.words('english'))
random_words = ["miss", "mr", "mrs", "said", "jr", "ms"]
nytimes_words = ["photo", "nytimes", "nytoday", "com", "www"]
custom_words = "___ saying hours told pareles news people like new going time think saturday _____ briefing posted eastern want email stories weekdays sundays rundown latest look sign let convention week american year say house york make know day late comedy host cbs report character guest night admission admissions charge fee johnson petrusich smith ratliff cover reservations reservation dunning tickets tonight broadway chinen meeting japanese moses sponsored island information state united bowery little last also life much each two many tomorrow free org th minimum little Sonneh feet street man st door play one would years get even could way back around avenue streets says go really made something still got came come right always sanneh percent nd near co la sisario go says east village advance clinton san hes cotter made wa ha harlem de west thing sold another still around called things city see well beach men women woman man"
month_time_dayofweek = "january february march april may june july august september october november december sunday monday tuesday wednesday thursday friday morning noon night weekday weekend evening mondays tuesdays wednesdays thursdays fridays saturdays sundays jan feb mar aprl jun jul aug sep oct nov dec mon tues weds thurs fri sat sun pm"
numbers_list = ["one", "two", "three", "four", "five", "six", "seven", "eight", "nine", "ten"]
custom_words_list = custom_words.split()
month_time_day_list = month_time_dayofweek.split()

STOPLIST = set(stopwords.words('english') + random_words + numbers_list + list(month_time_day_list) + list(ENGLISH_STOP_WORDS) + nytimes_words + custom_words_list)
STOPLIST = list(STOPLIST)

##  Create dataframe, taking each corpus document and appending it to a column called "Contents"
**Note:each article was saved as an individual txt document with the date and URL in the file name 

In [3]:
path = 'corpus/williamsburg_finalcorpus'

myfolder = os.listdir(path)
articles = []

slash = '/'

for filename in myfolder:
    if filename.endswith('.txt'):
        date = filename[0:10]
        with open(path + slash + filename) as fh:
            contents = fh.read()
            articles.append([contents, date])
            
raw = pd.DataFrame(articles, columns=['Contents','Date'])
print(raw)


                                               Contents        Date
0     EVEN people with faith in nothing, people who ...  2000-01-02
1     Correction Appended\nTENEMENTS, yes. Luxury ho...  2000-01-06
2     Two years ago, Joe, the man I lived with, sugg...  2000-01-14
3     IF they didn't play their luck out in furnitur...  2000-01-20
4     FOR years, Tequila's on Columbus held out a we...  2000-01-26
5     At Relish, a gleaming silver diner on Wythe St...  2000-02-06
6     Correction Appended\nGO out in Brooklyn? On th...  2000-02-11
7     BACK in the 60's, ''commune'' was often synony...  2000-02-20
8     Correction Appended\nA half a century after a ...  2000-02-21
9     The Real Form Girdle building, once a gritty f...  2000-02-27
10    Jane Pierce, manager of the Hamilton Fish Publ...  2000-03-14
11    Old-Fashioned Confections At Columbus Flea Mar...  2000-03-15
12    LIMOR TOMER, a Juilliard-trained pianist and f...  2000-03-26
13    THESE are among my favorite new restaurant

In [4]:
raw_no_duplicates = raw.drop_duplicates("Contents")
raw_no_duplicates

,Contents,Date
0,"EVEN people with faith in nothing, people who ...",2000-01-02
1,"Correction Appended\nTENEMENTS, yes. Luxury ho...",2000-01-06
2,"Two years ago, Joe, the man I lived with, sugg...",2000-01-14
3,IF they didn't play their luck out in furnitur...,2000-01-20
4,"FOR years, Tequila's on Columbus held out a we...",2000-01-26
5,"At Relish, a gleaming silver diner on Wythe St...",2000-02-06
6,Correction Appended\nGO out in Brooklyn? On th...,2000-02-11
7,"BACK in the 60's, ''commune'' was often synony...",2000-02-20
8,Correction Appended\nA half a century after a ...,2000-02-21
9,"The Real Form Girdle building, once a gritty f...",2000-02-27


Generate new rows for month and year and remove duplicates

In [5]:
raw_no_duplicates["Contents_Lower"] = raw_no_duplicates["Contents"].str.lower()
raw_no_duplicates["Contents_Clean"] = raw_no_duplicates["Contents_Lower"].apply(lambda x: x.replace("\n", " "))
raw_no_duplicates["Year"] = raw_no_duplicates["Date"].astype(str).str[0:4]
raw_no_duplicates["Month"] = raw_no_duplicates["Date"].astype(str).str[5:7]
raw_no_duplicates

C:\Users\Curri\Miniconda2\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\Users\Curri\Miniconda2\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\Curri\Miniconda2\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pand

,Contents,Date,Contents_Lower,Contents_Clean,Year,Month
0,"EVEN people with faith in nothing, people who ...",2000-01-02,"even people with faith in nothing, people who ...","even people with faith in nothing, people who ...",2000,01
1,"Correction Appended\nTENEMENTS, yes. Luxury ho...",2000-01-06,"correction appended\ntenements, yes. luxury ho...","correction appended tenements, yes. luxury hou...",2000,01
2,"Two years ago, Joe, the man I lived with, sugg...",2000-01-14,"two years ago, joe, the man i lived with, sugg...","two years ago, joe, the man i lived with, sugg...",2000,01
3,IF they didn't play their luck out in furnitur...,2000-01-20,if they didn't play their luck out in furnitur...,if they didn't play their luck out in furnitur...,2000,01
4,"FOR years, Tequila's on Columbus held out a we...",2000-01-26,"for years, tequila's on columbus held out a we...","for years, tequila's on columbus held out a we...",2000,01
5,"At Relish, a gleaming silver diner on Wythe St...",2000-02-06,"at relish, a gleaming silver diner on wythe st...","at relish, a gleaming silver diner on wythe st...",2000,02
6,Correction Appended\nGO out in Brooklyn? On th...,2000-02-11,correction appended\ngo out in brooklyn? on th...,correction appended go out in brooklyn? on the...,2000,02
7,"BACK in the 60's, ''commune'' was often synony...",2000-02-20,"back in the 60's, ''commune'' was often synony...","back in the 60's, ''commune'' was often synony...",2000,02
8,Correction Appended\nA half a century after a ...,2000-02-21,correction appended\na half a century after a ...,correction appended a half a century after a p...,2000,02
9,"The Real Form Girdle building, once a gritty f...",2000-02-27,"the real form girdle building, once a gritty f...","the real form girdle building, once a gritty f...",2000,02


Clean and preprocess text by: removing digits, stop words, and punctuation. Then tokenize words, stem them and lemmatize words for analysis.

In [42]:
import string
exclude = set(string.punctuation)

def is_good_utf8(str):
    try:
        str.decode("utf-8")
        return True
    except:
        return False
    
raw_no_duplicates["Contents_Clean"] = raw_no_duplicates["Contents_Clean"].apply(lambda x: ''.join([c for c in x if not c.isdigit()]))
raw_no_duplicates["Contents_nopunc"] = raw_no_duplicates["Contents_Clean"].apply(lambda x: ''.join([c for c in x if c not in exclude]))
raw_no_duplicates['Contents_Tokenized']=raw_no_duplicates["Contents_nopunc"].apply(lambda x : filter(is_good_utf8, filter(None,x.split(" "))))
raw_no_duplicates['Contents_decode']=raw_no_duplicates['Contents_Tokenized'].apply(lambda x: [word.decode('utf-8', ) for word in x])
raw_no_duplicates["Contents_Lemma"]=raw_no_duplicates["Contents_decode"].apply(lambda x : " ".join(lemma.lemmatize(word) for word in x))



C:\Users\Curri\Miniconda2\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [43]:
raw_no_duplicates

,Contents,Date,Contents_Lower,Contents_Clean,Year,Month,Contents_nopunc,Contents_Tokenized,Contents_decode,Contents_Lemma
0,"EVEN people with faith in nothing, people who ...",2000-01-02,"even people with faith in nothing, people who ...","even people with faith in nothing, people who ...",2000,01,even people with faith in nothing people who b...,"[even, people, with, faith, in, nothing, peopl...","[even, people, with, faith, in, nothing, peopl...",even people with faith in nothing people who b...
1,"Correction Appended\nTENEMENTS, yes. Luxury ho...",2000-01-06,"correction appended\ntenements, yes. luxury ho...","correction appended tenements, yes. luxury hou...",2000,01,correction appended tenements yes luxury housi...,"[correction, appended, tenements, yes, luxury,...","[correction, appended, tenements, yes, luxury,...",correction appended tenement yes luxury housin...
2,"Two years ago, Joe, the man I lived with, sugg...",2000-01-14,"two years ago, joe, the man i lived with, sugg...","two years ago, joe, the man i lived with, sugg...",2000,01,two years ago joe the man i lived with suggest...,"[two, years, ago, joe, the, man, i, lived, wit...","[two, years, ago, joe, the, man, i, lived, wit...",two year ago joe the man i lived with suggeste...
3,IF they didn't play their luck out in furnitur...,2000-01-20,if they didn't play their luck out in furnitur...,if they didn't play their luck out in furnitur...,2000,01,if they didnt play their luck out in furniture...,"[if, they, didnt, play, their, luck, out, in, ...","[if, they, didnt, play, their, luck, out, in, ...",if they didnt play their luck out in furniture...
4,"FOR years, Tequila's on Columbus held out a we...",2000-01-26,"for years, tequila's on columbus held out a we...","for years, tequila's on columbus held out a we...",2000,01,for years tequilas on columbus held out a welc...,"[for, years, tequilas, on, columbus, held, out...","[for, years, tequilas, on, columbus, held, out...",for year tequila on columbus held out a welcom...
5,"At Relish, a gleaming silver diner on Wythe St...",2000-02-06,"at relish, a gleaming silver diner on wythe st...","at relish, a gleaming silver diner on wythe st...",2000,02,at relish a gleaming silver diner on wythe str...,"[at, relish, a, gleaming, silver, diner, on, w...","[at, relish, a, gleaming, silver, diner, on, w...",at relish a gleaming silver diner on wythe str...
6,Correction Appended\nGO out in Brooklyn? On th...,2000-02-11,correction appended\ngo out in brooklyn? on th...,correction appended go out in brooklyn? on the...,2000,02,correction appended go out in brooklyn on the ...,"[correction, appended, go, out, in, brooklyn, ...","[correction, appended, go, out, in, brooklyn, ...",correction appended go out in brooklyn on the ...
7,"BACK in the 60's, ''commune'' was often synony...",2000-02-20,"back in the 60's, ''commune'' was often synony...","back in the 's, ''commune'' was often synonymo...",2000,02,back in the s commune was often synonymous wit...,"[back, in, the, s, commune, was, often, synony...","[back, in, the, s, commune, was, often, synony...",back in the s commune wa often synonymous with...
8,Correction Appended\nA half a century after a ...,2000-02-21,correction appended\na half a century after a ...,correction appended a half a century after a p...,2000,02,correction appended a half a century after a p...,"[correction, appended, a, half, a, century, af...","[correction, appended, a, half, a, century, af...",correction appended a half a century after a p...
9,"The Real Form Girdle building, once a gritty f...",2000-02-27,"the real form girdle building, once a gritty f...","the real form girdle building, once a gritty f...",2000,02,the real form girdle building once a gritty fa...,"[the, real, form, girdle, building, once, a, g...","[the, real, form, girdle, building, once, a, g...",the real form girdle building once a gritty fa...


In [9]:
len(raw_no_duplicates)

1477

In [60]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation


tfidf_vectorizer = TfidfVectorizer(max_df = 0.4, min_df = 15,
                                   stop_words = STOPLIST)
tfidf = tfidf_vectorizer.fit_transform(raw_no_duplicates["Contents_Lemma"])
tfidf_feature_names = tfidf_vectorizer.get_feature_names()
nmf = NMF(n_components = 10, random_state=1, alpha=.1, l1_ratio=.5).fit(tfidf)

In [55]:
#function to display the topics and terms that make up each topic for both models -- will be called at the end

def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic #%d:" % topic_idx)
        print(" ".join([feature_names[i]
                        for i in topic.argsort()[:-n_top_words - 1:-1]]))
    print()

In [61]:

#LDA Model
no_features = 1000

#specify number of topic words
no_top_words = 10
no_topics = 10

tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2, max_features=no_features, stop_words=STOPLIST)
tf = tf_vectorizer.fit_transform(raw_no_duplicates["Contents_Lemma"])
tf_feature_names = tf_vectorizer.get_feature_names()
lda = LatentDirichletAllocation(n_topics=no_topics, max_iter=5, learning_method='online', learning_offset=50.,random_state=0).fit(tf)




C:\Users\Curri\Miniconda2\lib\site-packages\sklearn\decomposition\online_lda.py:294: DeprecationWarning: n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21
  DeprecationWarning)


In [52]:
transformed_data = nmf.transform(tfidf)
len(transformed_data)

1477

In [62]:
print_top_words(nmf, tfidf_feature_names, 10)

Topic #0:
band song album rock guitar ballroom pop hall singer theater
Topic #1:
store friend business shop artist rent party apartment school company
Topic #2:
museum painting gallery artist exhibition sculpture photograph drawing image object
Topic #3:
dance ticket ballet opera theater program company choreographer dancer performance
Topic #4:
tour museum student child exhibition garden central walking corner hour
Topic #5:
jazz ticket band saxophonist song drummer pianist trio bassist quartet
Topic #6:
restaurant chef wine menu food dish beer diner opened burger
Topic #7:
post daily mayor bloomberg school police official authority officer department
Topic #8:
apartment developer housing development price unit project market million buyer
Topic #9:
theater preview musical production directs cast close stage drama revival
()


In [63]:
print_top_words(lda, tf_feature_names, no_top_words)


Topic #0:
store fashion designer style jean hair dress clothes bike vintage
Topic #1:
building school company development housing plan project brooklyn million neighborhood
Topic #2:
music art work apartment museum place month ticket dance brooklyn
Topic #3:
theater dance company center festival music film ballet musical performance
Topic #4:
ticket music dance work concert opera pianist quartet set program
Topic #5:
park tour museum child brooklyn student manhattan garden member exhibition
Topic #6:
team world cup game college event single gold bird national
Topic #7:
band music song album jazz rock brooklyn manhattan guitar pop
Topic #8:
brooklyn bar apartment building place neighborhood restaurant home manhattan space
Topic #9:
art show museum artist work gallery painting exhibition sculpture drawing
()
